In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/08 00:12:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/08 00:12:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/08 00:12:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
# creating a temporary table to use for sql queries
df_green.registerTempTable('green')

/home/gzuz/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")


In [5]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-28 19:00:00| 134|193.61000000000007|            17|
|2020-01-22 19:00:00|  65| 657.0300000000001|            41|
|2020-01-27 08:00:00|  17|             85.56|             4|
|2020-01-02 09:00:00|  66|229.39999999999998|            12|
|2020-01-02 12:00:00|  89|310.28000000000003|            14|
|2020-01-07 12:00:00|  66|             179.5|             9|
|2020-01-03 08:00:00| 223|165.90000000000003|             9|
|2020-01-17 10:00:00|  41| 638.2699999999999|            49|
|2020-01-02 11:00:00|  26|             198.6|             7|
|2020-01-12 20:00:00| 247|36.900000000000006|             3|
|2020-01-12 15:00:00|  10|             75.92|             1|
|2020-01-24 04:00:00| 129|148.89000000000001|            13|
|2020-01-04 20:00:00|  25| 369.5700000000001|            23|
|2020-01-22 10:00:00|  7

In [6]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode = 'overwrite')


In [7]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [8]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")


In [9]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-10 06:00:00| 144|            149.51|             9|
|2020-01-29 03:00:00|  13|            248.89|             6|
|2020-01-13 21:00:00| 255|            108.98|             8|
|2020-01-04 12:00:00| 249| 2245.569999999999|           152|
|2020-01-02 15:00:00| 236| 7382.910000000018|           504|
|2020-01-31 21:00:00|  50|1784.4399999999994|           104|
|2020-01-07 12:00:00|  68| 3046.229999999997|           181|
|2020-01-02 09:00:00| 161|4299.9299999999985|           242|
|2020-01-23 17:00:00| 233| 4642.769999999999|           235|
|2020-01-06 22:00:00| 158| 923.2100000000003|            57|
|2020-01-15 07:00:00|  13|2432.6299999999997|           118|
|2020-01-23 19:00:00| 224|            327.28|            16|
|2020-01-18 00:00:00| 144|2951.2999999999984|           174|
|2020-01-17 19:00:00| 17

In [10]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode = 'overwrite')

In [11]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')


In [12]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2021-05-19 10:00:00|  19|              14.3|             1|
|2020-12-20 13:00:00|  33|             17.55|             2|
|2020-08-23 13:00:00|  47|             27.86|             1|
|2020-09-10 10:00:00| 134|             27.91|             1|
|2020-03-31 11:00:00| 182|              20.3|             1|
|2020-02-05 19:00:00| 197|              34.1|             2|
|2020-02-18 08:00:00| 189|             66.64|             2|
|2020-01-07 16:00:00| 224|             90.36|             2|
|2020-02-18 08:00:00| 121|             32.36|             1|
|2020-03-09 05:00:00| 243|             51.82|             3|
|2020-09-29 09:00:00|  75|279.84000000000003|            12|
|2020-11-20 16:00:00| 225|             105.2|             3|
|2020-10-29 09:00:00| 150|             28.48|             1|
|2020-02-07 08:00:00| 16

In [13]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-03-31 23:00:00| 151|             30.35|             3|
|2020-10-28 07:00:00| 179|             58.29|             4|
|2020-11-23 13:00:00|  20|             29.19|             1|
|2020-08-08 01:00:00| 228|              93.5|             1|
|2020-05-08 07:00:00|  61| 84.13000000000001|             4|
|2020-05-03 20:00:00| 230|             78.37|             5|
|2020-01-26 02:00:00| 217|             57.51|             3|
|2020-08-17 23:00:00| 254|              57.3|             1|
|2020-08-08 12:00:00| 211|              65.0|             5|
|2021-05-26 07:00:00| 249| 978.3400000000001|            58|
|2020-10-05 13:00:00| 248|             42.29|             2|
|2021-06-21 20:00:00|  80|               0.0|             2|
|2021-02-04 09:00:00| 256|             17.01|             1|
|2021-04-08 17:00:00| 13

In [14]:
# rename amount and number_records to indicate which data is from which table
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [15]:
# outer join to include all rows from both tables including mismatched which will show "null"
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')


In [16]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  37|            175.67|                   6|161.60999999999999|                    7|
|2020-01-01 00:00:00|  40|168.97999999999996|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  45|  

In [17]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [18]:
# use materialized results rather than computing on the fly
df_join = spark.read.parquet('data/report/revenue/total')

In [19]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [20]:
df_zones = spark.read.parquet('zones/')

In [21]:
# join df_zones and df_zones on zone and locationid
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [22]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|        14| Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|        15|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|        17| Brooklyn|             Bedford|   Boro Zone|
|2020-01-01 00:00:00| 

In [24]:

df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones', mode='overwrite')

In [25]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|        14| Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|        15|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|        17| Brooklyn|             Bedford|   Boro Zone|
|2020-01-01 00:00:00| 